# Section 6. Working with spatial data

#### Instructor: Pierre Biscaye

The content of this notebook draws on material from my own research. 
    
### Learning Objectives 
    
* Think about what manipulations are needed to prepare raw spatial data for analysis
* Understand some of the common steps, such as interpolation and rescaling
* Be able to visualize point data together with other spatial data
* Think about ways to match spatial data to observation units for analysis

### Sections

1. Prepping spatial data for analysis
2. Point-level data analysis

In [ ]:
# Import Packages

import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd
import sys
import rasterio
from matplotlib.colors import LinearSegmentedColormap

%matplotlib inline

In [ ]:
# Setting my data path
#path="/Users/pierrebiscaye/Dropbox/Data/"
path="C:/Users/pibiscay/Dropbox/Data/"

# 1. Prepping data for analysis

In notebook 6a we walked through how to access and map a variety of remotely sensed and other spatial data products. But there is much more work to do before the data you access can be used for analysis.

We'll go through three types of data prep steps today:
1. Interpolating spatial data
2. Changing data resolution
3. Creating derived variables

We'll talk about these in the contex of code I used for my paper on the impacts of exposure to locust swarms on the risk of conflict.

In this paper, the unit of observation for the analysis was 0.25$^{\circ}$ cells at the annual level. This means that all my spatial data have to be prepared at this level. 

I first constructed a grid of 0.25$^{\circ}$ cells defined by cell centroids. This is what we have done previously in this class when creating meshed grids. I then duplicated this grid for the number of years in my dataset. The next step was to prepare my individual datasets to merge into this grid to create a master dataset for analysis.


### Interpolating spatial data

The GPW data are only available every 5 years. What to do if you want population data for the years in between?

One possibility is to **interpolate**, meaning to estimate missing data based on other observations.

For the case of the GPW data, the approach I took is **linear interpolation**, estimating population in years between observations as weighted averages of the two surrounding years with data. 

Below is the code I used to build a population array after loading the available data as above in Section 6a.

In [ ]:
# pop01=(4/5)*pop00.read(1) + (1/5)*pop05.read(1)
# pop02=(3/5)*pop00.read(1) + (2/5)*pop05.read(1)
# pop03=(2/5)*pop00.read(1) + (3/5)*pop05.read(1)
# pop04=(1/5)*pop00.read(1) + (4/5)*pop05.read(1)
# pop06=(4/5)*pop05.read(1) + (1/5)*pop10.read(1)
# pop07=(3/5)*pop05.read(1) + (2/5)*pop10.read(1)
# pop08=(2/5)*pop05.read(1) + (3/5)*pop10.read(1)
# pop09=(1/5)*pop05.read(1) + (4/5)*pop10.read(1)
# pop11=(4/5)*pop10.read(1) + (1/5)*pop15.read(1)
# pop12=(3/5)*pop10.read(1) + (2/5)*pop15.read(1)
# pop13=(2/5)*pop10.read(1) + (3/5)*pop15.read(1)
# pop14=(1/5)*pop10.read(1) + (4/5)*pop15.read(1)
# pop16=(4/5)*pop15.read(1) + (1/5)*pop20.read(1)
# pop17=(3/5)*pop15.read(1) + (2/5)*pop20.read(1)
# pop18=(2/5)*pop15.read(1) + (3/5)*pop20.read(1)
# pop19=(1/5)*pop15.read(1) + (4/5)*pop20.read(1)
# pop00=pop00.read(1)
# pop05=pop05.read(1)
# pop10=pop10.read(1)
# pop15=pop15.read(1)
# pop20=pop20.read(1)

# pop=np.array([pop00,pop01,pop02,pop03,pop04,
#              pop05,pop06,pop07,pop08,pop09,
#              pop10,pop11,pop12,pop13,pop14,
#              pop15,pop16,pop17,pop18,pop19,pop20])

The end result (after some additional preparation) was a dataset at the cell-year level that was ready to merge.

### Changing data resolution

#### Spatial resolution

When combining data sources it is necessary to match data resolution (as well as CRS!!).

Most of the data I used for my locusts and conflict project were at a finer resolution than 0.25$^{\circ}$ degrees. This means I had multiple pixels for each 0.25$^{\circ}$ grid cell. The approach here is to **rescale** the original data to match the desired resolution. You could see that I did this in the GEE script we looked at.

When the original data is at a higher resolution, you are **collapsing** it to the lower target resolution. You can take a mean, a sum, a max, or other statistics for higher resolution pixels to then generate statistics at the lower target resolution. What you use depends on what you are trying to measure.

When the original data is at a lower resolution, you are **spreading** the data across the lower target resolution. One approach is to assign all smaller pixels within the larger pixel the value of the larger pixel. More sophisticated approachs require assumptions about how the values in the larger pixel were generated, and using that as an input to determine how to spread the data. You might not want to do this for something like temperature, but you might for something like GDP or agricultural output.

#### Temporal resolution

Merging data sources also requires matching the temporal resolution.

How you approach this again depends on the relative resolutions of the original and target data sources. You can **collapse** data to larger temporal resolutions (i.e., months to years), or **spread** data across lower resolutions (i.e., years to days).

#### Example: Preparing WorldClim weather data

WorldClim has data on monthly total precipitation and maximum temperature available at a 2.5 arcminute resolution (around 0.04 degrees) globally for every month from 1985-2018. 

I needed to merge this 0.04$^{\circ}$ degree by month data to a 0.25$^{\circ}$ by year target dataset.

**Temporal resolution:** I took the sum of precipitation across months to get an annual measure and took the max of maximum monthly temperature. 

*Question:* Why did I use different methods for the two measures?

**Spatial resolution:** I assigned every original 0.04$^{\circ}$ degree cell to the larger grid cell it fell in, and took means across smaller cells to get an average value for the larger cells. 

*Question:* When might taking a sum or a max have been appropriate?

#### When to change resolution?

High-resolution spatial datasets take up a lot of memory, so it may be best to change the resolution to the desired level before saving the resulting dataset. In this case, you can write the code to change resolution in whatever codebook you are using to load and treat the raw data (EE JavaScript, Python, ...).

But saving higher-resolution datasets can be useful if you want to create flexibility for different types of merging later on. In this case, you can coarsen or collapse the data in the process of merging it in whatever your preferred econometric software is (Python, R, Stata, ...).

### Creating derived variables

#### In raster data

When working with rasters, you may want to create some derived variables in the original resolution as coarsening first and then creating variables in a combined dataset at the target resolution may lead to different values of derived variables.

You should think carefully about the math of what exactly the derived variables will be measuring depending on the order of operations and decide how to proceed.

*Question*: You have evapotranspiration and precipitation data at a monthly 25km resolution and want to calculate
$$ SPEI = norm(Precipitation - Evapotranspiration) $$
What would it imply to calculate this first at the monthly level and then derive an annual measure, as opposed to collapsing evapotranspiration and precipitation to the year level first and then trying to calculate SPEI?

#### Merging point data to a grid

In some cases you want to assign raster data to point locations, which we will discuss below. 

In other cases, you want to use information from point events to create a raster. This is what I did for the conflict and locust swarms data in my paper. 

I first identified which grid cell and year each point event was located in.

I then assigned values to cell-years based on the distribution of point events. I calculated two statistics for each cell-year:
* Count of events
* Indicators for any events

Finally, I created additional variables for each cell-year based on proximity to events outside of the cell, to consider potential spillovers.

### How did this all look in the end?

In [ ]:
clean="C:/Users/pibiscay/Dropbox/Locusts/Clean data/"
world=gpd.read_file(path+"Country boundaries/Country raw/"+
                    "UIA_World_Countries_Boundaries/World_Countries__Generalized_.shp")
data = pd.read_csv(clean+"mapping.csv")
x = data['lon']
y = data['lat']

In [ ]:
data.columns

In [ ]:
pop20=rasterio.open(path+"Spatial/GPW/2020/gpw_v4_population_count_rev11_2020_15_min.tif")
pop10=rasterio.open(path+"Spatial/GPW/2010/gpw_v4_population_count_rev11_2010_15_min.tif")
data['population']=data['population']*10000

In [ ]:
nodes = [0, 0.33, 0.67, 1]  # positions for each color from 0-1
color_scheme = ['bisque', 'orange', 'red', 'purple']  # corresponds to nodes
custom_cmap = LinearSegmentedColormap.from_list(
    'WhiteYellowRed', list(zip(nodes, color_scheme)))
custom_cmap.set_under('gray')  # set values under vmin to gray
custom_cmap.set_over('blue')  # set values over vmax to black

# Create a figure with three subplots
fig, ax = plt.subplots(3, 1, figsize=(10, 10))

title_fontsize = 20  # Size of the subplot titles
label_fontsize = 16  # Size of the colorbar labels
tick_fontsize = 12   # Size of the colorbar tick labels

im = ax[0].imshow(pop20.read(1),
               cmap=custom_cmap,
               extent=(-180, 180, -90, 90),
               vmin=0, vmax=500000)
ax[0].set_title('Population in 2020 (Source: CIESIN)', fontsize=title_fontsize)
cbar=fig.colorbar(im, ax=ax[0], label="Population")
cbar.ax.tick_params(labelsize=tick_fontsize) 
cbar.set_label("Population", fontsize=label_fontsize)

im = ax[1].imshow(pop10.read(1),
               cmap=custom_cmap,
               extent=(-180, 180, -90, 90),
               vmin=0, vmax=500000)
ax[1].set_title('Population in 2010 (Source: CIESIN)', fontsize=title_fontsize)
cbar=fig.colorbar(im, ax=ax[1], label="Population")
cbar.ax.tick_params(labelsize=tick_fontsize) 
cbar.set_label("Population", fontsize=label_fontsize)

scatter = ax[2].scatter(x, y, marker="s", s=10, c=data['population'], cmap=custom_cmap,
                       vmin=0, vmax=500000)
ax[2].set_title('Population in 2018 (Interpolated)', fontsize=title_fontsize)
cbar=fig.colorbar(scatter, ax=ax[2], label="Population")
cbar.ax.tick_params(labelsize=tick_fontsize) 
cbar.set_label("Population", fontsize=label_fontsize)

for i in range(3):
    world.plot(ax=ax[i],color='none', edgecolor='k', alpha=0.3)
    ax[i].set_xlim([30,50])
    ax[i].set_ylim([5,20])
plt.tight_layout()
plt.show()

In [ ]:
temp_18=rasterio.open(path+"/Spatial/WorldClim/Max Temperature/wc2.1_2.5m_tmax_2010-2018/"
                      +"wc2.1_2.5m_tmax_2018-07.tif")

In [ ]:
fig, ax = plt.subplots(2,1,figsize=(10, 10))
im = ax[0].imshow(temp_18.read(1),
               cmap='Reds',
               extent=(-180, 180, -90, 90),
               vmin=15, vmax=45)
ax[0].set_title('Max Temperature in July 2018 (Source: WorldClim)', fontsize=title_fontsize)
cbar=fig.colorbar(im, ax=ax[0], label="Temperature")
cbar.ax.tick_params(labelsize=tick_fontsize) 
cbar.set_label("Temperature (C)", fontsize=label_fontsize)

scatter = ax[1].scatter(x, y, marker="s", s=20, 
                        c=data['temp_max_july'], cmap='Reds', vmin=15, vmax=45)
ax[1].set_title("Max Temperature in July 2018, rescaled", fontsize=title_fontsize)
cbar=fig.colorbar(scatter, ax=ax[1], label="Temperature")
cbar.ax.tick_params(labelsize=tick_fontsize) 
cbar.set_label("Temperature (C)", fontsize=label_fontsize)

for i in range(2):
    world.plot(ax=ax[i],color='none', edgecolor='k', alpha=0.3)
    ax[i].set_xlim([-15,5])
    ax[i].set_ylim([10,25])
plt.tight_layout()
plt.show()
plt.show()

In [ ]:
swarms = pd.read_csv(path+"/Locust Hub/Retrieved 9.13.20/Swarms_geo_clean.csv")
swarms=swarms[swarms['yr']>1989] # because that is the focus in my paper

In [ ]:
swarms.columns

In [ ]:
# Color schemes
nodes = [0,  1] 
color_scheme3 = ['white', 'orange']  # corresponds to nodes
custom_cmap3 = LinearSegmentedColormap.from_list(
    'custom_name3', list(zip(nodes, color_scheme3)))
custom_cmap3.set_under('white')  # set values under vmin to white
custom_cmap3.set_over('orange')  # set values over vmax to blue

# Create a figure with three subplots
fig, axes = plt.subplots(3, 1, figsize=(10, 10))

title_fontsize = 20  # Size of the subplot titles
label_fontsize = 16  # Size of the colorbar labels
tick_fontsize = 12   # Size of the colorbar tick labels


scatter = axes[0].scatter(swarms['x'], swarms['y'], marker="s", s=1, c=swarms['yr'], cmap='jet')
axes[0].set_title("A. Swarm locations and year", fontsize=title_fontsize)
cbar=fig.colorbar(scatter, ax=axes[0], label="Year")
cbar.ax.tick_params(labelsize=tick_fontsize) 
cbar.set_label("Year", fontsize=label_fontsize)

scatter = axes[1].scatter(x, y, marker="s", s=1, c=data['treat_yr2'], cmap='jet')
axes[1].set_title("B. Year of first locust swarm exposure", fontsize=title_fontsize)
cbar=fig.colorbar(scatter, ax=axes[1], label="Year")
cbar.ax.tick_params(labelsize=tick_fontsize) 
cbar.set_label("Year", fontsize=label_fontsize)

scatter = axes[2].scatter(x, y, marker="s", s=1, c=data['swarm100max'], cmap=custom_cmap3)
axes[2].set_title("C. Any swarm within 100km", fontsize=title_fontsize)
cbar=fig.colorbar(scatter, ax=axes[2], ticks=[0, 1], label="0=No, 1=Yes")
cbar.ax.tick_params(labelsize=tick_fontsize) 
cbar.set_label("0=No, 1=Yes", fontsize=label_fontsize)

for i in range(3):
    world.plot(ax=axes[i],color='none', edgecolor='k', alpha=0.3)
    axes[i].set_xlim([-18,60])
    axes[i].set_ylim([-2,38])
plt.tight_layout()
plt.show()

# 2. Point-level analysis

We've looked at examples of using point data (on the location and timing of locust swarms) to create rasters. But in many cases we are interested in conducting analyses at the level of points - locations of individuals, cities, businesses, etc. 

There are a huge number of datasets with geolocated information on events, communities, survey locations, etc. It is often very useful to map these to other spatial data for analysis.

Let's look at two examples: locations of locust swarm observations from the [FAO Desert Locust Hub](https://locust-hub-hqfao.hub.arcgis.com/) and of survey communities in the Nigeria General Household Survey Panel ([GHSP](https://microdata.worldbank.org/index.php/catalog/5835)). 

### Plotting point data over a basemap

It can be useful to have a reference map under your data. You might want to plot terrain, streets, etc. This can help you better understand your point data. 

Here we will use the `Basemap` package to map relief/topography under data on the locations and dates of locust swarm observations. You will

In [ ]:
from mpl_toolkits.basemap import Basemap

In [ ]:
# Swarms
fig = plt.figure(figsize=(10, 10))
m = Basemap(projection='lcc',  
            lat_0=20, lon_0=-325,
            width=1.2E7, height=5.5E6)
m.drawcountries(color='gray')
m.shadedrelief()
m.scatter(swarms.x.values, swarms.y.values, latlon=True, 
          alpha=0.4, s=1, c=swarms.yr.values, cmap=plt.get_cmap("jet"))
plt.colorbar(label='Year',ticks=np.arange(1990, 2022, 4), orientation="horizontal",pad=0.01)
plt.show()

### Plotting point data over a raster

You may also want to plot your point data over raster data you have downloaded and that you might want to use for analysis. Let's plot GHSP community locations over a map of GAEZ agroecological zones.

In [ ]:
home = "C:/Users/pibiscay/Dropbox/Class-Data Science/Section 6/Data/"

In [ ]:
ghsp = pd.read_stata(home+"NGA_HouseholdGeovariables_Y1.dta", convert_categoricals=False)

In [ ]:
ghsp.columns

In [ ]:
gaez=rasterio.open(home+"faocmb_2010.tif")
nga_adm1 = gpd.read_file(home+"NGA_adm1.shp")

In [ ]:
fig, ax = plt.subplots(ncols=1, figsize=(10,10))
nga_adm1.plot(ax=ax, color='none', edgecolor='k', label="State boundaries")
ax.scatter(ghsp['lon_dd_mod'],ghsp['lat_dd_mod'], marker='+', c='red')
im = ax.imshow(gaez.read(1),
               cmap='viridis',
               extent=(-180, 180, -90, 90))
fig.colorbar(im)
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.set_xlim([2,15])
ax.set_ylim([4,14])
ax.set_title('GHSP 2010-11 survey locations and 2000 cell cropland share')
plt.show()

### Geospatial calculations

Now that we've brought together data sources, we can use them to conduct calculations.

For example, for a given location we might want to match data on weather and geography.

Let's do an example of calculating distance to the nearest body of water for community locations in the GHSP.

In [ ]:
nga_wat_area = gpd.read_file(home+"NGA_water_areas_dcw.shp")

In [ ]:
fig, ax = plt.subplots(ncols=1, figsize=(10,10))
nga_adm1.plot(ax=ax, color='none', edgecolor='k', label="State boundaries")
nga_wat_area.plot(ax=ax, color='blue', edgecolor='b', alpha=0.1, label="Bodies of water")
ax.scatter(ghsp['lon_dd_mod'],ghsp['lat_dd_mod'], color='r', marker='+',alpha=0.6)
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.set_title('GHSP 2010-11 survey locations and Nigeria water bodies')
plt.show()

In [ ]:
type(ghsp)

In [ ]:
# First turn the survey locations into a geopandas gdf
from shapely.geometry import Point
ghsp_geo=ghsp
ghsp_geo['Coordinates'] = list(zip(ghsp.lon_dd_mod, ghsp.lat_dd_mod))
ghsp_geo['Coordinates'] = ghsp_geo['Coordinates'].apply(Point)
ghsp_geo = gpd.GeoDataFrame(ghsp_geo, geometry='Coordinates')
ghsp_geo=ghsp_geo.set_crs(epsg=4326)

In [ ]:
type(ghsp_geo)

In [ ]:
type(nga_wat_area)

In [ ]:
nga_wat_area.geometry

In [ ]:
nga_wat_area.crs==ghsp_geo.crs
# If this were not true, we could use the geopandas to_crs method 
# to reproject one GeoDataFrame to match the CRS of the other.

In [ ]:
nga_wat_area.crs

#### Reprojecting

In a geographic CRS (e.g., EPSG:4326, WGS 84), distances are measured in degrees, which are not consistent or accurate for calculating real-world distances. For example, 1 degree of longitude covers different distances depending on the latitude.

We need to pick a projected CRS, where the units are typically in meters, suitable for your region. A common choice is the Universal Transverse Mercator (UTM) projection.
Nigeria is covered by multiple UTM zones: EPSG:32631, EPSG:32632, and EPSG:32633. We can use any of these based on the specific location of the data, or we can use a more general projected CRS such as EPSG:3857 (Web Mercator) for simplicity.

To resolve this, we need to re-project both GeoDataFrames to a projected CRS before performing the distance calculation. This requires the `pyproj` library - you will need to install it.

You can run the following code in your command interface (not the one running this notebook), or use the Anaconda Navigator to install these libraries to your environment.
`conda install anaconda::pyproj`
`conda install conda-forge::proj`

In [ ]:
projected_crs = "EPSG:32633"
ghsp_geo = ghsp_geo.to_crs(projected_crs)
nga_wat_area = nga_wat_area.to_crs(projected_crs)

Now we can use the `shapely.distance` method to calculate distances. It will calculate pairwise distances, and we will take the minimum.

In [ ]:
# Calculate the distance to the nearest water body for each point in ghsp_geos
def calculate_nearest_distance(point, polygons):
    """
    Calculate the distance from a point to the nearest polygon in a GeoDataFrame.
    """
    return polygons.distance(point).min()

In [ ]:
# Apply the function to calculate distances; this can take a while!
ghsp_geo['distance_to_water'] = ghsp_geo['Coordinates'].apply(lambda x: calculate_nearest_distance(x, nga_wat_area['geometry']))

In [ ]:
# Check the resulting summary stats
ghsp_geo['distance_to_water'].describe()

In [ ]:
# Reproject back
ghsp_geo = ghsp_geo.to_crs("EPSG:4326")
nga_wat_area = nga_wat_area.to_crs("EPSG:4326")

In [ ]:
# Now let's map it!
fig, ax = plt.subplots(ncols=1, figsize=(10,10))
nga_adm1.plot(ax=ax, color='none', edgecolor='k', label="State boundaries")
nga_wat_area.plot(ax=ax, color='blue', edgecolor='b', alpha=0.1, label="Bodies of water")
im=ax.scatter(ghsp_geo['lon_dd_mod'],ghsp_geo['lat_dd_mod'], 
              c=ghsp_geo['distance_to_water'], marker='s', cmap='viridis')
fig.colorbar(im)
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.set_title('GHSP 2010-11 survey locations and distance to nearest water')
plt.show()


### Analysis at the point level

In my paper on the impacts of the 2012 floods in Nigeria, I determine flooding status of each community according to both survey reports and satellite imagery, as processed by the NASA Near-Real Time Flood Mapping Product.

The two definitions disagree. I analyze the spatial characteristics of different categories of community to try to identify predictors of differences in flood identification. This was the impetus for the project I discussed earlier comparing flood mapping sources. 

SHOW THE SLIDES.

